In [1]:
from sudoku import Sudoku
import numpy as np
import argparse
import cv2 as cv
from torch.nn.modules.activation import Softmax
import torch.nn.functional as F
import torch.nn as nn
import torch
from collections import OrderedDict
from skimage.segmentation import clear_border
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2 as cv



# # Start capturing the video
# cap = cv2.VideoCapture(0)

# if __name__ == "__main__":
#     detected = False
#     solved = False

In [2]:
# Hyperparameters for our network
input_size   = 784
hidden_sizes = [128, 64]
output_size  = 10

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.dropout1 = nn.Dropout(p = 0.25)
        self.dropout2 = nn.Dropout(p = 0.50)
        self.fc1 = nn.Sequential(
			nn.Linear(7*7*64, 64),
			nn.ReLU())
        self.fc2 = nn.Sequential(
			nn.Linear(64, 10),
			nn.ReLU())
            
    def forward(self, input):
        out = self.layer1(input)
        out = self.layer2(out)
        out = self.dropout1(out)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        out = self.dropout2(out)
        out = self.fc2(out)
        out = F.softmax(out, dim = 1)
        return out

In [3]:
model = Network()
model.load_state_dict(torch.load('output/best_model.pt'))
model.eval()

Network(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Sequential(
    (0): Linear(in_features=3136, out_features=64, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
    (1): ReLU()
  )
)

In [2]:
from imutils.perspective import four_point_transform

In [6]:
def preprocessImage(image, skip_dilation = False):
	preprocess = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
	preprocess = cv.adaptiveThreshold(preprocess, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 101, 2)

	if not skip_dilation:
		kernel = np.array([[0., 1., 0.], [1., 2., 1.], [0., 1., 0.]], dtype = np.uint8)
		preprocess = cv.dilate(preprocess, kernel)
	return preprocess

In [3]:
def find_puzzle(image, debug=False):
    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (1, 1), 0)
    
    thresh = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    thresh = cv.bitwise_not(thresh)

    # if debug:
    #     cv.imshow("Puzzle Thresh", thresh)
    #     cv.waitKey(0)

    # finding countours in the image and sort by descending size
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv.contourArea, reverse=True)

    puzzleCnt = None

    for c in cnts:
        # approximate the contour
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.02 * peri, True)

        # if our approximated contour has four points, then we
        # can assume that we have found our puzzle
        if len(approx) == 4:
            puzzleCnt = approx
            break

    if puzzleCnt is None:
        raise Exception("Could not find Sudoku puzzle.")

    	
    # apply a four point perspective transform to both the original
	# image and grayscale image to obtain a top-down bird's eye view
	# of the puzzle
    puzzle = four_point_transform(image, puzzleCnt.reshape(4, 2))
    warped = four_point_transform(gray, puzzleCnt.reshape(4, 2))
	# check to see if we are visualizing the perspective transform
    if debug:
		# show the output warped image (again, for debugging purposes)
        cv.imshow("Puzzle Transform", warped)
        cv.waitKey(0)
	# return a 2-tuple of puzzle in both RGB and grayscale
    return (puzzle, warped)

In [4]:
img = cv.imread("resources/images/Sudoku4.png")
image = imutils.resize(img, width=600)

(puzzleImage, warped) = find_puzzle(image, debug=False)



# initialize our 9x9 Sudoku board
board = np.zeros((9, 9), dtype="int")
# a Sudoku puzzle is a 9x9 grid (81 individual cells), so we can
# infer the location of each cell by dividing the warped image
# into a 9x9 grid
stepX = warped.shape[1] // 9
stepY = warped.shape[0] // 9
# initialize a list to store the (x, y)-coordinates of each cell
# location
cellLocs = []
board_list = []

In [9]:
def preProcessing(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.equalizeHist(img)
    img = img/255
    return img

In [13]:
def extract_digit(cell, debug=False):
	# apply automatic thresholding to the cell and then clear any
	# connected borders that touch the border of the cell
    thresh = cv.threshold(cell, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
    thresh = clear_border(thresh)
	# check to see if we are visualizing the cell thresholding step
    if debug:
        cv.imshow("Cell Thresh", thresh)
        cv.waitKey(0)
    
    # find contours in the thresholded cell
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
	# if no contours were found than this is an empty cell
    if len(cnts) == 0:
        return None
	# otherwise, find the largest contour in the cell and create a
	# mask for the contour
    c = max(cnts, key=cv.contourArea)
    mask = np.zeros(thresh.shape, dtype="uint8")
    cv.drawContours(mask, [c], -1, 255, -1)
    
    (h, w) = thresh.shape
    percentFilled = cv.countNonZero(mask) / float(w * h)
    if percentFilled < 0.03:
        return None

    digit = cv.bitwise_and(thresh, thresh, mask=mask)

    if debug:
        # show the digit extracted from the cell
        cv.imshow("Digit", digit)
        cv.waitKey(0)

    return digit

In [27]:
# load the digit classifier from disk
print("[INFO] loading digit classifier...")
model = load_model('output/digit_classifier.h5')
# load the input image from disk and resize it
print("[INFO] processing image...")
image = cv.imread('resources/images/Sudoku3.png')
image = imutils.resize(image, width=600)

[INFO] loading digit classifier...
[INFO] processing image...


In [28]:
(puzzleImage, warped) = find_puzzle(image, debug=False)
# initialize our 9x9 Sudoku board
board = np.zeros((9, 9), dtype="int")
# a Sudoku puzzle is a 9x9 grid (81 individual cells), so we can
# infer the location of each cell by dividing the warped image
# into a 9x9 grid
stepX = warped.shape[1] // 9
stepY = warped.shape[0] // 9
# initialize a list to store the (x, y)-coordinates of each cell
# location
cellLocs = []

In [13]:
# loop over the grid locations
for y in range(0, 9):
	# initialize the current list of cell locations
    row = []
    row_board = []
    for x in range(0, 9):
		# compute the starting and ending (x, y)-coordinates of the
		# current cell
        startX = x * stepX
        startY = y * stepY
        endX = (x + 1) * stepX
        endY = (y + 1) * stepY
		# add the (x, y)-coordinates to our cell locations list
        row.append((startX, startY, endX, endY))

        # crop the cell from the warped transform image and then
		# extract the digit from the cell
        cell = warped[startY:endY, startX:endX]

        digit = extract_digit(cell, debug=True)
        
		# verify that the digit is not empty
        if digit is not None:
			# resize the cell to 28x28 pixels and then prepare the
			# cell for classification

            roi = cv.resize(digit, (28, 28))
            roi = torch.Tensor(roi).unsqueeze(dim=0).unsqueeze(dim=0)

            with torch.no_grad():
                prediction = model(roi)
                prediction = torch.argmax(prediction, dim=1).item()
                print(prediction)


#         else:
#             row_board.append(0)

# 	# add the row to our cell locations
#     cellLocs.append(row)
#     board_list.append(row_board)

# print(board_list)

8
8
1
8
3
8
4
8
8
2
3
8
8
2
8
8
7
2
8
7
8
3
8
1


In [32]:
# loop over the grid locations
for y in range(0, 9):
	# initialize the current list of cell locations
	row = []
	for x in range(0, 9):
		# compute the starting and ending (x, y)-coordinates of the
		# current cell
		startX = x * stepX
		startY = y * stepY
		endX = (x + 1) * stepX
		endY = (y + 1) * stepY
		# add the (x, y)-coordinates to our cell locations list
		row.append((startX, startY, endX, endY))
        # crop the cell from the warped transform image and then
		# extract the digit from the cell
		cell = warped[startY:endY, startX:endX]
		digit = extract_digit(cell, debug=False)
		# verify that the digit is not empty
		if digit is not None:
			# resize the cell to 28x28 pixels and then prepare the
			# cell for classification
			roi = cv.resize(digit, (28, 28))
			roi = roi.astype("float") / 255.0
			roi = img_to_array(roi)
			roi = np.expand_dims(roi, axis=0)
			# classify the digit and update the Sudoku board with the
			# prediction
			pred = model.predict(roi).argmax(axis=1)[0]
			board[y, x] = pred
	# add the row to our cell locations
	cellLocs.append(row)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

In [30]:
# construct a Sudoku puzzle from the board
print("[INFO] OCR'd Sudoku board:")
puzzle = Sudoku(3, 3, board=board.tolist())
puzzle.show()
# solve the Sudoku puzzle
print("[INFO] solving Sudoku puzzle...")
solution = puzzle.solve()
solution.show_full()

[INFO] OCR'd Sudoku board:
+-------+-------+-------+
| 8     |   1   |     9 |
|   5   | 8   7 |   1   |
|     4 |   9   | 7     |
+-------+-------+-------+
|   6   | 7   1 |   2   |
| 5   8 |   6   | 1   7 |
|   1   | 5   2 |   9   |
+-------+-------+-------+
|     7 |   4   | 6     |
|   8   | 3   9 |   4   |
| 3     |   5   |     8 |
+-------+-------+-------+

[INFO] solving Sudoku puzzle...

---------------------------
9x9 (3x3) SUDOKU PUZZLE
Difficulty: SOLVED
---------------------------
+-------+-------+-------+
| 8 7 2 | 4 1 3 | 5 6 9 |
| 9 5 6 | 8 2 7 | 3 1 4 |
| 1 3 4 | 6 9 5 | 7 8 2 |
+-------+-------+-------+
| 4 6 9 | 7 3 1 | 8 2 5 |
| 5 2 8 | 9 6 4 | 1 3 7 |
| 7 1 3 | 5 8 2 | 4 9 6 |
+-------+-------+-------+
| 2 9 7 | 1 4 8 | 6 5 3 |
| 6 8 5 | 3 7 9 | 2 4 1 |
| 3 4 1 | 2 5 6 | 9 7 8 |
+-------+-------+-------+

        


In [13]:
board = board_list

puzzle = Sudoku(3, 3, board=board)

print(puzzle)


---------------------------
9x9 (3x3) SUDOKU PUZZLE
Difficulty: INVALID PUZZLE
---------------------------
+-------+-------+-------+
|       | 4     |       |
|       |   5   |       |
|       |       |     9 |
+-------+-------+-------+
|   2   |       |       |
| 1     |       |       |
|       |       | 7     |
+-------+-------+-------+
|       |       |   8   |
|       |     6 |       |
|     3 |       |       |
+-------+-------+-------+

        


In [14]:
puzzle.solve().show_full()


---------------------------
9x9 (3x3) SUDOKU PUZZLE
Difficulty: SOLVED
---------------------------
+-------+-------+-------+
| 7 9 5 | 4 8 2 | 1 3 6 |
| 6 1 8 | 9 5 3 | 2 4 7 |
| 2 3 4 | 6 7 1 | 8 5 9 |
+-------+-------+-------+
| 8 2 7 | 1 3 5 | 9 6 4 |
| 1 4 9 | 7 6 8 | 3 2 5 |
| 3 5 6 | 2 4 9 | 7 1 8 |
+-------+-------+-------+
| 4 6 1 | 3 9 7 | 5 8 2 |
| 9 8 2 | 5 1 6 | 4 7 3 |
| 5 7 3 | 8 2 4 | 6 9 1 |
+-------+-------+-------+

        
